In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys; sys.path.insert(0, '..') # add parent folder path where lib folder is

In [3]:
# from ml.preprocess import data
import pickle
from dask_ml.preprocessing import StandardScaler
from dask_ml.model_selection import train_test_split

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score

import numpy as np
import pandas as pd

from utils import helper, config, dasker

from ml.models.base.slug_ann import SlugANN
from ml.models.base.slug_xgboost import SlugXGBoost
from ml.models.base.brisk_xgboost import BriskXGBoost
from ml.models.ensemble import BaseUnify


In [4]:
client = dasker.get_local_client()
# cl =  dasker.get_global_client()
print(client)

2022-10-26 09:28:27,784 - asyncio - DEBUG - Using selector: EpollSelector


2022-10-26 09:28:28,815 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-nlw39gh6', purging
2022-10-26 09:28:28,816 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-hy24l55h', purging
2022-10-26 09:28:28,816 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-zy9r1jz5', purging
2022-10-26 09:28:28,816 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-xa_9drh5', purging


<Client: 'tcp://127.0.0.1:8786' processes=4 threads=4, memory=15.37 GiB>


In [5]:
df_X, df_y = helper.get_covid_dataset()
df_X = df_X.drop(['location'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.33, random_state=config.rand_state)

X_scalar = StandardScaler().fit_transform(df_X.copy())

X_base_scalar, X_test_scalar, y_base_scalar, y_test_scalar = train_test_split(X_scalar, df_y, test_size=0.33, random_state=config.rand_state)
X_train_scalar, X_val_scalar, y_train_scalar, y_val_scalar = train_test_split(X_base_scalar, y_base_scalar, random_state=config.rand_state)


In [7]:
slug_ann = SlugANN()
slug_ann.pred_class = 'regression'
slug_ann.X_train = X_train_scalar
slug_ann.X_test = X_test_scalar
slug_ann.X_val = X_val_scalar
slug_ann.y_train = y_train_scalar
slug_ann.y_test = y_test_scalar
slug_ann.y_val = y_val_scalar
slug_ann.n_trials = 300

slug_xgb = SlugXGBoost()
slug_xgb.pred_class = 'regression'
slug_xgb.objective = 'count:poisson'
slug_xgb.X_train = X_train
slug_xgb.X_test = X_test
slug_xgb.y_train = y_train
slug_xgb.y_test = y_test
slug_xgb.n_trials = 300

base_models = [slug_ann, slug_xgb]

In [8]:
config.create_project_dirs(overwrite=False)

Directory structure for project /covid created successfully


In [9]:
def __timer_expired__(sig, frame):
    print('\n')
    # print(f'timer of {timer}s expired')    
    print('Ensemble training session interrupted due to timer runout')
    
    client.shutdown()
    unified_models.fetch_models(retrain=False)    

In [23]:
unified_models = BaseUnify(timer=20, base_models=base_models, callback_timer_expired=None)
unified_models.ensemble_n_trials = 200


In [ ]:
unified_models.fetch_models(retrain=True)

2022-10-26 09:41:22,295 - console_info - INFO - ensemble: base models discovery started
2022-10-26 09:41:22,295 - console_info - INFO - ensemble: base models discovery started
2022-10-26 09:41:23,111 - console_info - INFO - slug xgboost: Starting train for trials:300 with boosted rounds: 250
2022-10-26 09:41:23,111 - console_info - INFO - slug xgboost: Starting train for trials:300 with boosted rounds: 250
2022-10-26 09:41:23,120 - console_info - INFO - slug xgboost: Cleared previous models in the model save path
2022-10-26 09:41:23,120 - console_info - INFO - slug xgboost: Cleared previous models in the model save path
2022-10-26 09:41:23,201 - console_info - INFO - slug ann: Starting train for trials:300 with epochs: 300
2022-10-26 09:41:23,201 - console_info - INFO - slug ann: Starting train for trials:300 with epochs: 300
2022-10-26 09:41:23,210 - console_info - INFO - slug ann: Cleared previous models in the model save path
2022-10-26 09:41:23,210 - console_info - INFO - slug ann:

In [27]:
unified_models.fit_ensemble(retrain=True)

2022-10-26 09:41:16,829 - console_info - INFO - xgboost: no saved models found, please rerun the 'fetch_model' first.
2022-10-26 09:41:16,829 - console_info - INFO - xgboost: no saved models found, please rerun the 'fetch_model' first.
2022-10-26 09:41:16,834 - console_info - INFO - xgboost: no saved models found, please rerun the 'fetch_model' first.
2022-10-26 09:41:16,834 - console_info - INFO - xgboost: no saved models found, please rerun the 'fetch_model' first.
2022-10-26 09:41:16,848 - console_info - INFO - xgboost: no saved models found, please rerun the 'fetch_model' first.
2022-10-26 09:41:16,848 - console_info - INFO - xgboost: no saved models found, please rerun the 'fetch_model' first.
2022-10-26 09:41:16,870 - console_info - INFO - xgboost: no saved models found, please rerun the 'fetch_model' first.
2022-10-26 09:41:16,870 - console_info - INFO - xgboost: no saved models found, please rerun the 'fetch_model' first.
2022-10-26 09:41:16,880 - console_info - INFO - xgboost:

In [ ]:
unified_models.best_base_model

In [ ]:
pred = unified_models.predict(df_X)

In [ ]:
r2_score(pred, df_y)

In [ ]:
pred = unified_models.predict(X_train)
r2_score(pred, y_train)

In [ ]:
pred = unified_models.predict(X_test)
r2_score(pred, y_test)